In [7]:
## imports
import pandas as pd
from fastai import *
from fastai.text import * 
import numpy as np
import matplotlib.pyplot as plt

from google.colab import files

AttributeError: ignored

In [6]:
## upload data
uploaded = files.upload()

NameError: ignored

In [ ]:
X, Y = [], []

count = 0

dataList = uploaded['training.data'].decode('cp1252').split('\n')
for line in dataList:
    if count == len(dataList)-1:
        break
    y, x = line.split(' ', 1)
    X.append(x)
    Y.append(y)
    count += 1

In [ ]:
data = pd.DataFrame({
    'text':X,
    'labels':Y})

data = data.sample(frac=1)
data = data.reset_index(drop=True)
print(len(data))
data.head()

In [ ]:
print(data['labels'].value_counts())
plt.hist(data['labels'])
plt.xticks(rotation=90, fontsize=5)
plt.show()

In [ ]:
split = 0.85
cutoff = int(len(data) * split)

#Loading only few training and validation samples, for quick training time
trn_texts = data.loc[0:cutoff, 'text'].values
trn_labels = data.loc[0:cutoff, 'labels'].values
val_texts = data.loc[cutoff:len(data)-1, 'text'].values
val_labels = data.loc[cutoff:len(data)-1, 'labels'].values
np.random.seed(42)
trn_idx = np.random.permutation(len(trn_texts))
val_idx = np.random.permutation(len(val_texts))
trn_texts = trn_texts[trn_idx]
val_texts = val_texts[val_idx]
trn_labels = trn_labels[trn_idx]
val_labels = val_labels[val_idx]

col_names = ['labels','text']
df_trn = pd.DataFrame({'text':trn_texts, 'labels':trn_labels}, columns=col_names)
df_val = pd.DataFrame({'text':val_texts, 'labels':val_labels}, columns=col_names)

In [ ]:
data_lm = TextLMDataBunch.from_df('./', train_df=df_trn, valid_df=df_val)
learner = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.6)

In [ ]:
learner.fit_one_cycle(10, 2e-2, moms=(0.8, 0.7))

In [ ]:
learner.save_encoder('my_enc')

In [ ]:
# Classifier model data
data_clas = TextClasDataBunch.from_df('./', train_df=df_trn, valid_df=df_val, vocab=data_lm.train_ds.vocab, bs=32)
classifier = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5)
classifier.load_encoder('my_enc')

In [ ]:
classifier.fit_one_cycle(10, 2e-2, moms=(0.8, 0.7))

In [ ]:
classifier.recorder.plot_losses()

In [ ]:
classifier.show_results(rows=1000)

In [ ]:
prediction = classifier.predict("What does RDM stand for?")[1].item()
data_clas.classes

In [ ]:
prediction = classifier.predict("Block out 10 mins where you practice holding a handstand no wall assist for as long as you can record your best time.")[1].item()
data_clas.classes[prediction]

In [ ]:
prediction = classifier.predict("The rock_paper_scissors_multiagent.py example demonstrates several types of policies competing against each other.")[1].item()
data_clas.classes[prediction]

In [ ]:
prediction = classifier.predict("Why is the world so bad?")[1].item()
data_clas.classes[prediction]